#### **Basic instructions:**
#### 1. **To run code cells**, you can either hit the play button to the left of the cell, or you can select the cell and **press shift-enter**.

#### 2. **The first time** you run this code notebook, you might get a popup asking to choose which version of Python to use (the python "kernel"). **Just hit enter** to choose the base/default version.

#### 3. Make sure you data (.abf) files are in the "data" folder here on the left. You can just copy/paste the files over from where they are saved on your computer.

In [ ]:
from glob import glob
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from utils import *
from single_channel_analysis import *
update_plot_defaults()

%load_ext autoreload
%autoreload 2

# Single channel recordings

## 1. Choose the data file you want to analyze

#### Put the .abf files with your recordings in the "data/Single_channels" folder

In [ ]:
pathToData = "data/Single_channels"
abf_files = glob(pathToData+"/*.abf")
abf_files

Choose which file you want to analyze and past the file name here:

In [ ]:
data_file = 'data/Single_channels/2024_06_11_0003.abf'

Now we can load the file and plot the raw data:

In [ ]:
data_file = 'data/Single_channels/2024_06_11_0003.abf'

## Extract the sweeps, time and sampling frequency:
sweeps, sweeps_time, sampling_freq = get_sweeps(data_file)

print(f"Data dimensions (sweeps, channels, timepoints): \n{sweeps.shape}")
print(f"sampling freq: {sampling_freq} Hz")

current_traces = sweeps[:, 0, :]
voltage_traces = sweeps[:, 1, :]
ax = plot_traces(sweeps_time*1000, current_traces, voltage_traces)
plt.show()

# Highpass filter the current traces


## 2. Select what you want to measure

### 2.1. Defining the time window

First let's define the relevant time window where we want to measure the currents:

(If you want to save the images, you can do it by clicking on the save icon in the top-right of the image when you hover over it)

In [ ]:
# The start/end time just helps to zoom in on the traces.
start_time = 35  # seconds
end_time = 75   # seconds

current_traces, time = select_sweep_window(sweeps, sweeps_time, start_time*1000, end_time*1000, sampling_freq, channel=0)
time /= 1000 # Convert to Seconds

fig, ax = plt.subplots(figsize=(15, 6), linewidth=0.1)
ax.plot(time, current_traces[0], color='k')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Current (pA)')
plt.show()

### 2.2. Lowpass filter the current trace

In [ ]:
# Median baseline subtraction
window_size_ms = 4000
corrected_median, baseline_median = baseline_correction(current_traces, sampling_freq, method='median_subtraction', window_ms=window_size_ms)

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 5), sharex=True)

# Original
axes[0].plot(time, current_traces.T, alpha=0.9, color='black')
axes[0].set_ylabel('Current (pA)')
axes[0].set_title('Original Traces')
axes[0].grid(True, alpha=0.3)

# Median subtraction
axes[1].plot(time, corrected_median.T, alpha=0.9, color='black')
axes[1].set_ylabel('Current (pA)')
axes[1].set_title(f'Median Subtraction ({window_size_ms/1000}s window)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show baseline estimation for median method
fig, ax = plt.subplots(figsize=(12, 6))
trace_idx = 0  # Show first trace
ax.plot(time, current_traces[trace_idx], 'b-', alpha=0.7, label='Original')
ax.plot(time, baseline_median[trace_idx], 'r-', linewidth=2, label='Estimated Baseline')
ax.plot(time, corrected_median[trace_idx], 'k-', alpha=0.9, label='Corrected')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Current (pA)')
ax.set_title('Baseline Estimation Example')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### 2.3. Highpass filter the currrent trace

In [ ]:
# Highpass filtering
cutoff = 1000
baseline_corrected = corrected_median 

filtered_traces = lowpass_filter(baseline_corrected, cutoff, sampling_freq, filter_type='bessel')

# Plot filter comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 5), sharex=True)
axes[0].plot(time, baseline_corrected.T, alpha=0.9, color='black')
axes[0].set_title('Baseline Corrected (No Lowpass)')
axes[0].set_ylabel('Current (pA)')
axes[0].grid(True, alpha=0.3)
axes[1].plot(time, filtered_traces.T, alpha=0.9, color='black')
axes[1].set_title(f'Bessel {cutoff} Hz')
axes[1].set_ylabel('Current (pA)')
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2.3. Detect events (channel openings)

In [ ]:
# Specify number of levels (including baseline)
n_conductance_levels = 2  # Adjust based on your channels: baseline + open levels

# Detect levels automatically
detected_levels, fit_info = detect_levels_from_histogram(filtered_traces, n_conductance_levels, plot_result=True, bins=150)

# The baseline will be the lowest detected level
baseline_level = detected_levels[0]
current_levels_above_baseline = detected_levels[1:]  # Levels above baseline

print(f"\nBaseline level: {baseline_level:.3f} pA")
print(f"Levels above baseline: {[f'{level:.3f}' for level in current_levels_above_baseline]} pA")

In [ ]:
# Initialize detector with your filtered traces
time_seconds = time / 1000  # Convert time from milliseconds to seconds
detector = MultiLevelEventDetector(filtered_traces, time_seconds, sampling_freq)

detector.baseline_level = baseline_level
detector.set_current_levels(current_levels_above_baseline)
detector.level_names

In [ ]:
# Set detection parameters
detector.min_event_duration = 2.0  # ms - adjust based on your channel kinetics
detector.hysteresis_factor = 0.15  # Helps with noisy transitions

# Test on a single trace first
events_trace1, idealized_trace1 = detector.detect_events_single_trace(0, plot_result=True)

# ---------------------------------------------------------------------
# Print the results
print(f"Found {len(events_trace1)} events in trace 1")
if events_trace1:
    print("First few events:")
    for i, event in enumerate(events_trace1[:5]):
        start_ms, end_ms, level_idx, amplitude = event
        level_name = detector.level_names[level_idx-1] if level_idx > 0 else 'Baseline'
        duration_ms = (end_ms - start_ms) * 1000
        print(f"  Event {i+1}: {start_ms*1000:.1f}-{end_ms*1000:.1f} ms, {level_name}, {duration_ms:.2f} ms duration")
print(f"\nTo process all traces, run: detector.detect_all_events()")
print(f"To adjust levels, run: detector.set_current_levels([level1, level2, ...])")


In [ ]:
events_all, idealized_traces = detector.detect_all_events(plot_summary=True, duration_hist_bins='auto')

In [ ]:
# Get all event durations
durations = detector.calculate_event_durations()
# Returns: {'L1': [dur1, dur2, ...], 'L2': [dur1, dur2, ...], ...}

# Plot duration histograms with exponential fitting
durations = detector.plot_duration_histogram(fit_exponential=True,
                                             bins='auto',
                                             separate_plots=False) # True = separate plot for each level

In [ ]:
# Calculate open probability
po_overall, po_by_level, statistics = detector.calculate_open_probability()

print(f"Overall open probability: {po_overall:.4f} ({po_overall*100:.2f}%)")
# po_by_level gives you Po for each individual level
# statistics gives you total recording time, open time, etc.

In [ ]:
# Generate comprehensive report
detector.generate_analysis_report()
# Prints everything: Po, mean durations, event counts, etc.